# Preliminary Steps for T5 with Separate Attention Layer

## Preprocessing

- **Text Cleaning**: Clean the text data, removing any irrelevant characters, correcting formatting issues, and standardizing text for better model performance.

- **Tokenization**: Convert text data into a format suitable for the T5 model, typically using a tokenizer specific to T5.

- **Data Splitting**: Split the dataset into training, validation, and testing sets.

In [ ]:
!pip install transformers

In [ ]:
import pandas as pd
from transformers import T5Tokenizer
from sklearn.model_selection import train_test_split

# Load the dataset
data = pd.read_csv('NYT_Dataset.csv')

# Data Cleaning Function
def clean_text(text):
    if pd.isna(text):
        return ""  # Return an empty string if the text is NaN
    # Strip leading/trailing whitespace, replace newline characters, and other potential cleaning steps
    return text.strip().replace("\n", " ")

data['abstract'] = data['abstract'].apply(clean_text)
data['title'] = data['title'].apply(clean_text)

# Initialize T5 Tokenizer
tokenizer = T5Tokenizer.from_pretrained('t5-small')

# Function to tokenize text
def tokenize_texts(text):
    # Encode the texts using the T5 tokenizer. You might want to adjust the max_length depending on your specific needs
    return tokenizer.encode(text, max_length=512, truncation=True, return_tensors="pt")

# Tokenize abstracts (inputs) and titles (targets)
data['tokenized_abstracts'] = data['abstract'].apply(tokenize_texts)
data['tokenized_titles'] = data['title'].apply(tokenize_texts)

# Split the data into training, validation, and test sets
train, dev = train_test_split(data, test_size=0.2, random_state=42)
validation, test = train_test_split(dev, test_size=0.5, random_state=42)

# Display the structure of the split data
print(f"Training Set: {len(train)} samples")
print(f"Validation Set: {len(validation)} samples")
print(f"Test Set: {len(test)} samples")

# Save the cleaned dataset
columns_to_save = ['ID', 'title', 'topic', 'abstract', 'Date', 'keywords']
data[columns_to_save].to_csv('cleaned_NYT_Dataset.csv' , index=False)
print(f"Dataset saved as {'cleaned_NYT_Dataset.csv'}")



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Training Set: 85204 samples
Validation Set: 10651 samples
Test Set: 10651 samples
Dataset saved as cleaned_NYT_Dataset.csv


## Word2Vec Embedding for Keywords

In [ ]:
import gensim.downloader as api
import numpy as np

# Load pre-trained Word2Vec embeddings
word_vectors = api.load('word2vec-google-news-300')  # 300-dimensional vectors

# Function to get embeddings for a keyword
def get_embedding(keyword):
    try:
        return word_vectors[keyword]
    except KeyError:
        return np.zeros(300)  # Return a zero vector if keyword not in vocabulary


[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:
def get_keyword_ids(keywords, max_keywords, keyword_to_id):
    keyword_ids = [keyword_to_id.get(keyword, 0) for keyword in keywords.split(',')[:max_keywords]]
    if len(keyword_ids) < max_keywords:
        keyword_ids += [0] * (max_keywords - len(keyword_ids))  # Padding with 0 (assuming 0 is the index for unknown keywords)
    return keyword_ids

In [ ]:
import pandas as pd

unique_keywords = set()
for index, row in data.iterrows():
    if pd.notna(row['keywords']):  # Check if the 'keywords' value is not NaN
        unique_keywords.update(row['keywords'].split(','))
keyword_to_id = {keyword: i for i, keyword in enumerate(unique_keywords)}

## Initialization of T5 Model, Tokenizer, and Device Setup

- Category and Keyword Embedding
- Integration in Data Preparation



In [ ]:
import pandas as pd

# Check for unique categories
unique_categories = data['topic'].unique()  # Replace 'category_column_name' with the actual column name
num_categories = len(unique_categories)
print(f"Number of unique categories: {num_categories}")

# Check for unique keywords
# Assuming keywords are stored in a single column separated by commas
all_keywords = set()
data['keywords'].dropna().str.split(',').apply(all_keywords.update)  # Replace 'keywords_column_name' with the actual column name
num_keywords = len(all_keywords)
print(f"Number of unique keywords: {num_keywords}")


Number of unique categories: 3
Number of unique keywords: 48268


In [ ]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from transformers import T5ForConditionalGeneration, T5Tokenizer
from torch.nn.utils.rnn import pad_sequence

# Load the tokenizer and the T5 model
tokenizer = T5Tokenizer.from_pretrained('t5-small')
t5_model = T5ForConditionalGeneration.from_pretrained('t5-small')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
t5_model.to(device)

class T5CustomDataset(Dataset):
    def __init__(self, data, tokenizer, keyword_to_id, max_len=512, max_keywords=10):
        self.tokenizer = tokenizer
        self.data = data
        self.keyword_to_id = keyword_to_id
        self.max_len = max_len
        self.max_keywords = max_keywords

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data.iloc[idx]
        input_text = f"summarize: {item['abstract']}"
        target_text = item['title']

        input_ids = self.tokenizer.encode(input_text, truncation=True, padding="max_length", max_length=self.max_len, return_tensors="pt").squeeze()
        labels = self.tokenizer.encode(target_text, truncation=True, padding="max_length", max_length=self.max_len, return_tensors="pt").squeeze()

        # Handle keywords using Word2Vec embedding directly
        keywords = item['keywords'].split(',')[:self.max_keywords]
        keyword_ids = [self.keyword_to_id.get(kw, 0) if kw in self.keyword_to_id else 0 for kw in keywords]

        return {
            'input_ids': input_ids,
            'labels': labels,
            'keyword_ids': torch.tensor(keyword_ids, dtype=torch.long),
            'attention_mask': input_ids.ne(self.tokenizer.pad_token_id).int()
        }

def collate_fn(batch):
    input_ids = [item['input_ids'] for item in batch]
    labels = [item['labels'] for item in batch]
    attention_masks = [item['attention_mask'] for item in batch]
    keyword_ids = [item['keyword_ids'] for item in batch]

    input_ids = pad_sequence(input_ids, batch_first=True, padding_value=tokenizer.pad_token_id)
    labels = pad_sequence(labels, batch_first=True, padding_value=tokenizer.pad_token_id)
    attention_masks = pad_sequence(attention_masks, batch_first=True, padding_value=0)

    max_keywords = max(len(kw) for kw in keyword_ids)  # Find the max number of keywords in any entry
    keyword_ids_padded = torch.zeros((len(keyword_ids), max_keywords), dtype=torch.long)  # Initialize a zero tensor
    for i, kws in enumerate(keyword_ids):
        keyword_ids_padded[i, :len(kws)] = kws  # Assign without wrapping in torch.tensor()

    return {
        'input_ids': input_ids,
        'labels': labels,
        'attention_mask': attention_masks,
        'keyword_ids': keyword_ids_padded
    }


batch_size = 16

train_loader = DataLoader(T5CustomDataset(train, tokenizer, keyword_to_id, 512, 10), batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
validation_loader = DataLoader(T5CustomDataset(validation, tokenizer,  keyword_to_id, 512, 10), batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
test_loader = DataLoader(T5CustomDataset(test, tokenizer, keyword_to_id, 512, 10), batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

class ModifiedT5Model(nn.Module):
    def __init__(self, t5_model, num_keywords, emb_dim):
        super(ModifiedT5Model, self).__init__()
        self.t5_model = t5_model
        self.attribute_embeddings = nn.Embedding(num_keywords, emb_dim)
        self.special_attention = nn.Linear(emb_dim * 2, emb_dim)

    def forward(self, input_ids, attention_mask, labels, keyword_ids):
        input_embeddings = self.t5_model.get_input_embeddings()(input_ids)
        keyword_embeddings = self.attribute_embeddings(keyword_ids)

        keyword_embeddings_mean = keyword_embeddings.mean(dim=1)
        expanded_keyword_embeddings = keyword_embeddings_mean.unsqueeze(1).expand(-1, input_ids.size(1), -1)
        combined_embeddings = torch.cat((input_embeddings, expanded_keyword_embeddings), dim=-1)

        enhanced_embeddings = self.special_attention(combined_embeddings)

        outputs = self.t5_model(inputs_embeds=enhanced_embeddings, attention_mask=attention_mask, labels=labels)

        # Check if 'loss' is in the outputs, and if so, return it properly
        if 'loss' in outputs:
            return outputs.loss, outputs.logits
        else:
            # Return None or appropriate structure if 'loss' is not calculated
            return None, outputs.logits


# Assuming num_keywords and emb_dim are defined based on your dataset and embedding size
num_keywords = 48268  # Example, adjust based on actual count
emb_dim = 512  # Embedding dimension to match the T5 model's hidden size
modified_t5_model = ModifiedT5Model(t5_model, num_keywords, emb_dim).to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

# Training & Evaluation

- **Embedding Handling**: The training loop now retrieves category and keyword embeddings and combines them. This combined embedding is then used to enhance the text input embeddings.

- **Model Input Adjustments**: Depending on how your model is designed to accept input embeddings, you might need to modify the model.forward method or how the embeddings are passed to the model. This example simply adds the extra embeddings to the text embeddings for demonstration. In practice, you might want to integrate them more seamlessly, perhaps through a custom layer or attention mechanism as discussed previously.

- **Checkpointing and Monitoring**: The training progress and checkpointing remain the same, ensuring you can monitor the training process and recover or start from checkpoints if necessary.

## Metric Evaluation

In [ ]:
!pip install tqdm

In [ ]:
!pip install nltk rouge

In [ ]:
import nltk
from rouge import Rouge
from nltk.translate.bleu_score import corpus_bleu

# Function to convert ids to text
def ids_to_text(ids, tokenizer):
    return tokenizer.decode(ids, skip_special_tokens=True)

# Function to calculate BLEU and ROUGE scores
def calculate_metrics(references, hypotheses):
    # BLEU
    bleu_score = corpus_bleu([[ref.split()] for ref in references], [hyp.split() for hyp in hypotheses])

    # ROUGE
    rouge = Rouge()
    rouge_score = rouge.get_scores(hypotheses, references, avg=True)

    return bleu_score, rouge_score

## Training Loop

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm
import torch

# Assume 'model' is an instance of 'ModifiedT5Model' which has been modified to take 'keyword_ids'
model = modified_t5_model  # Ensure this is your modified model instance that takes keyword embeddings
optimizer = AdamW(model.parameters(), lr=5e-5, weight_decay=0.01)

# Training parameters
epochs = 2
batch_size = 16
best_val_loss = float('inf')
patience_counter = 0
patience = 2

scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_loader) * epochs)

# Training loop
model.train()
for epoch in range(epochs):
    total_loss = 0
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch + 1}")
    for batch in progress_bar:
        optimizer.zero_grad()

        # Move data to the appropriate device
        input_ids = batch['input_ids'].to(device)
        labels = batch['labels'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        keyword_ids = batch['keyword_ids'].to(device)  # Ensure keyword_ids are included

        # Forward pass through the model
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels, keyword_ids=keyword_ids)
        print(f"Outputs: {outputs}")
        loss = outputs.loss
        loss.backward()  # Backpropagation
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()
        progress_bar.set_postfix(loss=f"{total_loss / (progress_bar.n + 1):.2f}")

    # Save a checkpoint at the end of each epoch
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }, f'checkpoint_epoch_{epoch}.pth')

print(f"Training completed with total loss: {total_loss}")

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 1:   0%|          | 0/5326 [00:01<?, ?it/s]


AttributeError: 'tuple' object has no attribute 'loss'

In [ ]:
import time

def validate_and_calculate_scores(model, val_loader, tokenizer):
    model.eval()
    total_loss = 0
    references = []
    hypotheses = []
    start_time = time.time()  # Start timing

    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['target_ids'].to(device)

            # Generate outputs
            outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=50, no_repeat_ngram_size=2, early_stopping=True)
            loss = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels).loss
            total_loss += loss.item()

            for label, output in zip(labels, outputs):
                decoded_label = tokenizer.decode(label, skip_special_tokens=True).strip()
                decoded_output = tokenizer.decode(output, skip_special_tokens=True).strip()
                if decoded_label:
                    references.append(decoded_label)
                    hypotheses.append(decoded_output)

    evaluation_time = time.time() - start_time  # End timing
    print(f"Evaluation took {evaluation_time:.2f} seconds")

    if not references or not hypotheses:
        raise ValueError("No valid data found for evaluation. Please check your dataset.")

    try:
        bleu, rouge = calculate_metrics(references, hypotheses)
    except ValueError as e:
        print(f"Error calculating metrics: {e}")
        bleu, rouge = 0, 0

    return total_loss / len(val_loader), bleu, rouge, evaluation_time

# Run evaluation independently
val_loss, bleu_score, rouge_scores, eval_time = validate_and_calculate_scores(model, validation_loader, tokenizer)
print(f"Validation Loss: {val_loss:.2f}, BLEU Score: {bleu_score:.2f}, ROUGE Scores: {rouge_scores}, Evaluation Time: {eval_time:.2f} seconds")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:535: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


Evaluation took 752.10 seconds
Validation Loss: 0.10, BLEU Score: 0.04, ROUGE Scores: {'rouge-1': {'r': 0.14992050048996028, 'p': 0.16448521266811175, 'f': 0.15155953941855835}, 'rouge-2': {'r': 0.04639685677939932, 'p': 0.048947169171747275, 'f': 0.04621111149126243}, 'rouge-l': {'r': 0.14223244361393195, 'p': 0.15563314778249285, 'f': 0.14361149524313763}}, Evaluation Time: 752.10 seconds


In [ ]:
def display_input_output_pairs(model, data_loader, tokenizer, attribute_embeddings, category_map, keyword_map, num_pairs=5):
    model.eval()
    batch = next(iter(data_loader))  # Get one batch from the DataLoader

    # Prepare category and keyword embeddings
    category_ids = batch['category_ids'].to(device)
    keyword_ids = batch['keyword_ids'].to(device)
    category_emb, keyword_emb = attribute_embeddings(category_ids, keyword_ids)
    extra_emb = category_emb + keyword_emb

    # Prepare input embeddings
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['labels'].to(device)
    text_embeddings = model.get_input_embeddings()(input_ids)

    # Combine text and extra embeddings and use for generating outputs
    enhanced_text_embeddings = text_embeddings + extra_emb.unsqueeze(1).expand(-1, text_embeddings.size(1), -1)
    outputs = model.generate(inputs_embeds=enhanced_text_embeddings, attention_mask=attention_mask, max_length=50, no_repeat_ngram_size=2, early_stopping=True)

    print("Displaying Input-Output Pairs:\n")
    for i in range(min(num_pairs, len(batch['input_ids']))):
        input_text = tokenizer.decode(input_ids[i], skip_special_tokens=True)
        generated_title = tokenizer.decode(outputs[i], skip_special_tokens=True)
        reference_title = tokenizer.decode(labels[i], skip_special_tokens=True)

        # Decode category and keywords for display
        category_name = category_map[category_ids[i].item()]
        keywords_names = [keyword_map[kw_id] for kw_id in keyword_ids[i] if kw_id in keyword_map]

        print(f"Input Abstract {i+1}: {input_text}")
        print(f"Generated Title {i+1}: {generated_title}")
        print(f"Reference Title {i+1}: {reference_title}")
        print(f"Category: {category_name}")
        print(f"Keywords: {', '.join(keywords_names)}\n")

# Assuming model, validation_loader, tokenizer, attribute_embeddings, category_map, and keyword_map are already defined and set up
display_input_output_pairs(model, validation_loader, tokenizer, attribute_embeddings, category_map, keyword_map, num_pairs=5)
